In [124]:
# importing libraries
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

In [125]:
death_df = pd.read_csv('https://raw.githubusercontent.com/OliverArthur/covid-19-rd-data/master/death.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/OliverArthur/covid-19-rd-data/master/confirmed.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/OliverArthur/covid-19-rd-data/master/recovered.csv')
provinces_df = pd.read_csv('https://raw.githubusercontent.com/OliverArthur/covid-19-rd-data/master/provinces.csv')
case_per_provinces_df = pd.read_csv('https://raw.githubusercontent.com/OliverArthur/covid-19-rd-data/master/case-per-provinces.csv')

In [126]:
# clean data
death_df.columns = map(str.lower, death_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)
provinces_df.columns = map(str.lower, provinces_df.columns)

In [127]:
confirmed_df.head()
cols = confirmed_df.keys()
confirmed = confirmed_df.loc[:, cols[3]:cols[-1]]
deaths = death_df.loc[:, cols[3]:cols[-1]]
recovered_list = recovered_df.loc[:, cols[3]:cols[-1]]

dates = confirmed.keys()
confirmed_total = []
deaths_total = []
recovered_total = []
for i in dates:
    confirmed_sum = confirmed[i].sum()
    deaths_sum = deaths[i].sum()
    recovered_sum = recovered_list[i].sum()
    confirmed_total.append(confirmed_sum)
    deaths_total.append(deaths_sum)
    recovered_total.append(recovered_sum)
    

In [128]:
# number of provicens to enter
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    y = 'background-color: purple'
    g = 'background-color: grey'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = y
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    
    return df1

def show_latest_cases(n):
    n = int(n)
    return case_per_provinces_df.sort_values('confirmados', ascending= False).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [129]:
display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: #fff; font-size:30px;'> Confirmados: "  + str(confirmed_sum) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Muertos: " + str(deaths_sum) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recuperados: " + str(recovered_sum) + "</span>"+
             "</div>")
       )



In [130]:
sorted_provinces_df = case_per_provinces_df.sort_values('confirmados', ascending= False)

In [131]:
# # plotting the 10 worst hit provinces
def bubble_chart(n):
    fig = px.scatter(sorted_provinces_df.head(n), x="provincia", y="confirmados", size="confirmados", color="provincia",
               hover_name="provincia", size_max=60)
    fig.update_layout(
    title=str(n) +" Las provincias más afectados",
    xaxis_title="Provincias",
    yaxis_title="Casos confirmados",
    width = 700
    )
    fig.show()

interact(bubble_chart, n=10)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

    'data': [], 'layout': {'autosize': True, 'template': '...'}
}),), layout=Lay…

In [132]:
px.bar(
    sorted_provinces_df.head(10),
    x = "provincia",
    y = "confirmados",
    title= "Top 10 provincias mas afectadas", # the axis names
    color_discrete_sequence=["cyan"], 
    height=500,
    width=800
)

In [133]:
px.bar(
    sorted_provinces_df.head(10),
    x = "provincia",
    y = "muertos",
    title= "Top 10 provincias con mas muertes", # the axis names
    color_discrete_sequence=["red"], 
    height=500,
    width=800
)

In [134]:
px.bar(
    sorted_provinces_df.head(10),
    x = "provincia",
    y = "recuperados",
    title= "Top 10 provincias con mas casos de recuperados", # the axis names
    color_discrete_sequence=["green"], 
    height=500,
    width=800
)

In [123]:
RD_COORDINATES = (18.606258, -70.106722)
provinces_map = folium.Map(location=RD_COORDINATES, tiles="cartodbpositron", zoom_start=6, max_zoom=24, min_zoom = 2)

for i in range(0, len(case_per_provinces_df)):
    folium.Circle(
        location=[case_per_provinces_df.loc[i]['lat'], case_per_provinces_df.loc[i]['long']],
        fill=True,
        radius=(int((np.log(case_per_provinces_df.iloc[i]['confirmados']+1.00001)))+0.2)*4000,
        color='red',
        fill_color='indigo',
        tooltip="<div style='margin: 0; background-color: black; color: white;'>"+
            "<h4 style='text-align:center;font-weight: bold'>"+case_per_provinces_df.iloc[i]['provincia'] + "</h4>"
            "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
            "<li>Confirmed: "+str(case_per_provinces_df.iloc[i]['confirmados'])+"</li>"+
                        "<li>Deaths:   "+str(case_per_provinces_df.iloc[i]['muertos'])+"</li>"+
                    "</ul></div>",
        
    ).add_to(provinces_map)

provinces_map